In [1]:
import Base: iterate,max, exp, sin, cos, tan, +, ^, -, *, /, sqrt, convert, promote_rule, zero,isless
using BenchmarkTools

# Define dual number

In [2]:
struct Dual{T <: Number} <: Number
    x::T
    dx::T
end
Dual(n::Integer, d::Float64) = Dual(promote(n, d)...)
Dual(n::Float64, d::Integer) = Dual(promote(n, d)...)

Dual

In [3]:
function convert(::Type{Dual{T}}, x::T) where {T}
   Dual(x, zero(x)) 
end
function convert(::Type{Dual{T}}, x::Dual{S}) where {S, T}
    Dual(T(x.x), T(x.dx))
end
function convert(::Type{Dual{T}}, x::T) where {T <: Number}
    Dual(x, zero(x))
end
function convert(::Type{Dual{T}}, x::S) where {T, S <: Number}
    x_as_T = convert(T, x)
    Dual(x_as_T, zero(x_as_T))
end

convert (generic function with 191 methods)

In [4]:
function zero(x::Dual{T}) where T
    Dual(zero(x.x), zero(x.dx))
end

zero (generic function with 23 methods)

In [5]:
function promote_rule(::Type{Dual{T}}, ::Type{Dual{S}}) where {T,S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{Dual{T}}, ::Type{S}) where {T, S <: Number}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{T}, ::Type{Dual{S}}) where {T <: Number, S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{S}, ::Type{Dual{T}}) where {S <: AbstractIrrational, T}
    Dual{promote_type(S,T)}
end

promote_rule (generic function with 128 methods)

In [6]:
function extract_derivative(xdx::Dual)
    return xdx.dx
end
function extract_derivative(xs::Array)
    [extract_derivative(x) for x in xs]
end
function extract_derivative(xs::Tuple)
    convert(Tuple, [extract_derivative(x) for x in xs])
end

extract_derivative (generic function with 3 methods)

# Differential operator

In [7]:
function derivativeСalculation(f, value)
    function df(x)
        xdx = Dual(x, one(x))
        result = f(xdx)
        return extract_derivative(result)
    end
    df(value)
end
function derivativeСalculation(f,i::Integer,values)
    function df(valuesS)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(valuesS)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end
function derivativeСalculation(f,values)
    function df(xs...)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(xs)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end

derivativeСalculation (generic function with 2 methods)

In [8]:
function +(x::Dual, y::Dual)
    Dual(x.x+y.x, x.dx+y.dx)
end
function -(x::Dual, y::Dual)
    Dual(x.x-y.x, x.dx-y.dx)
end
function -(x::Dual)
    Dual(-x.x, -x.dx)
end
function *(x::Dual, y::Dual)
     Dual(x.x*y.x, x.x*y.dx + x.dx*y.x)
end
function /(x::Dual, y::Dual)
    Dual(x.x/y.x, x.dx/y.x - x.x*y.dx/(y.x*y.x))
end
function /(x::Array, y::Number)
    return [i / y for i in x]
end

function ^(a::Dual, x::Dual)
    Dual(a.x^x.x, a.dx * x.x * a.x ^ (x.x - 1) + x.dx * a.x ^ x.x * log(a.x)) 
end

function ^(a::Dual, x::Integer)
    Dual(a.x^x, a.dx * x * a.x ^ (x - 1)) 
end

^ (generic function with 68 methods)

In [9]:
function sqrt(x::Dual)
    Dual(sqrt(x.x), x.dx/(2*sqrt(x.x)))
end

sqrt (generic function with 20 methods)

In [10]:
function exp(x::Dual)
    return Dual(exp(x.x), exp(x.x)*x.dx)
end
function exp(xs::Array)
    print(xs)
    return [exp(x) for x in xs]
end
function sin(x::Dual)
    return Dual(sin(x.x), cos(x.x)*x.dx)
end
function cos(x::Dual)
    return Dual(cos(x.x), -sin(x.x)*x.dx)
end

function tan(x::Dual)
    return Dual(tan(x.x),(1/cos(x.x)^2)*x.dx)
end
isless(x::Dual, y::Dual) = x.x < y.x;

In [11]:
function max(x::Dual)
    return Dual(max(0,x.x), x.x < 0 ? 0 : 1 * x.dx)
end
function relu(x)
    return max(0,x)
end

relu (generic function with 1 method)

In [12]:
function softmax(vector::Array)
    e = exp(vector)
    return e / sum(e)
end

softmax (generic function with 1 method)

In [13]:
function iterate(iter::Vector{Dual}, state=1)
    if state > length(iter)
        return nothing
    end
    return (iter[state],state+1)
end

iterate (generic function with 228 methods)

In [14]:
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
                Dual(args[j], one(args[j])) :
                Dual(args[j],zero(args[j])) )
        end
        temp  = [f(x)...]
        column = extract_derivative.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

## Definicja funkcji testowych

In [15]:
function styblinskiTang(x)
    value = zero(x[1])
    for i=1:length(x)
        value += x[i]^4 - 16 * x[i]^2 + 5 * x[i]
    end
    value / 2
end

function rosenbrock(x)
    value = zero(x[1])
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end




rosenbrock (generic function with 1 method)

In [16]:
function TestFunction1(x)
    f1 = 123.83*(x[2]^8)^2 + cos(x[1]^18) - sin(x[1])/exp(x[2])
end

function TestFunction2(x)
    f1 = 222.83*(x[2]^8+x[1]^-3)^2 + sin(x[1]^18) - cos(x[2])/exp(x[1])
    f2 = x[1] - 5/x[2] + 4*x[1]^2-2*x[2] / (x[2]*sin(x[2])^2 + 1)
    [f1,f2]
end

function viennet(x)
    f1 = 0.5*(x[1]^2+x[2]^2) + sin(x[1]^2+ x[2]^2)
    f2 = (x[1]-2*x[2]+4)^2/8 + (x[1]-x[2]+1)^2/27 + 15
    f3 = 1/(x[1]^2+x[2]^2+1) - 1.1* exp(-(x[1]^2+x[2]^2))
    [f1,f2,f3]
end

function TestFunction4(x)
    f1 = 222.83*(x[2]^8+x[1]^-3)^2 + sin(x[1]^18) - cos(x[2])/exp(x[1])
    f2 = (x[1]^15-234.32)^3 + (tan(x[2]^5) + sin(x[1])^98)^36 
    f3 = x[1]*x[2]^34 - tan(343)^2 - sin(x[1])^8
    f4 = (x[1]^2+x[2]-4325.43)^23 - cos(x[2])*sin(x[1])^3
    [f1,f2,f3,f4]
end

function TestFunction5(x)
    f1 = 222.83*(x[2]^8+x[1]^-3)^2 + sin(x[1]^18) - cos(x[2])/exp(x[1])
    f2 = x[1]+x[1]^232+(sqrt(x[2]))^2+ (x[1]+x[2]^3)/233
    f3 = 2342434+23*x[1]^32-tan(x[2])+ (x[2]*sin(x[2])^2 + 1)
    f4 = 4*x[1]^2-2*x[2] / (x[2]*sin(x[2])^2 + 1)
    f5 = x[1] - 5/x[2] + 4*x[1]^2-2*x[2]
    [f1,f2,f3,f4,f5]
end

TestFunction5 (generic function with 1 method)

## Testy funkcji weluzmiennych 

### Testowanie funkcji Rosenbrock

In [17]:
J(rosenbrock, [5.3,2.2,34.2,123.1])

1×4 Matrix{Float64}:
 54895.4  -31012.4  1.43226e7  -209308.0

In [18]:
@benchmark J(rosenbrock, [rand(5,1)...])

BenchmarkTools.Trial: 
  memory estimate:  4.11 KiB
  allocs estimate:  48
  --------------
  minimum time:     2.536 μs (0.00% GC)
  median time:      2.608 μs (0.00% GC)
  mean time:        2.957 μs (8.86% GC)
  maximum time:     356.988 μs (99.01% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [19]:
@benchmark J(rosenbrock, [rand(50,1)...])

BenchmarkTools.Trial: 
  memory estimate:  130.81 KiB
  allocs estimate:  561
  --------------
  minimum time:     123.666 μs (0.00% GC)
  median time:      124.954 μs (0.00% GC)
  mean time:        132.211 μs (3.40% GC)
  maximum time:     2.657 ms (94.37% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [20]:
@benchmark J(rosenbrock, [rand(100,1)...])

BenchmarkTools.Trial: 
  memory estimate:  467.36 KiB
  allocs estimate:  1212
  --------------
  minimum time:     483.390 μs (0.00% GC)
  median time:      492.306 μs (0.00% GC)
  mean time:        523.619 μs (3.41% GC)
  maximum time:     3.883 ms (81.74% GC)
  --------------
  samples:          9496
  evals/sample:     1

In [21]:
@benchmark J(rosenbrock, [rand(500,1)...])

BenchmarkTools.Trial: 
  memory estimate:  8.20 MiB
  allocs estimate:  7014
  --------------
  minimum time:     11.296 ms (0.00% GC)
  median time:      11.564 ms (0.00% GC)
  mean time:        11.934 ms (1.91% GC)
  maximum time:     14.618 ms (15.65% GC)
  --------------
  samples:          419
  evals/sample:     1

In [22]:
@benchmark J(rosenbrock, [rand(1000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  32.11 MiB
  allocs estimate:  15018
  --------------
  minimum time:     44.910 ms (0.00% GC)
  median time:      46.268 ms (1.75% GC)
  mean time:        46.528 ms (1.42% GC)
  maximum time:     50.295 ms (2.07% GC)
  --------------
  samples:          108
  evals/sample:     1

In [23]:
@benchmark J(rosenbrock, [rand(5000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  1.23 GiB
  allocs estimate:  90024
  --------------
  minimum time:     1.147 s (1.40% GC)
  median time:      1.149 s (1.37% GC)
  mean time:        1.149 s (1.37% GC)
  maximum time:     1.151 s (1.35% GC)
  --------------
  samples:          5
  evals/sample:     1

In [24]:
@benchmark J(rosenbrock, [rand(10000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  4.89 GiB
  allocs estimate:  190025
  --------------
  minimum time:     4.624 s (1.38% GC)
  median time:      4.637 s (1.43% GC)
  mean time:        4.637 s (1.43% GC)
  maximum time:     4.651 s (1.49% GC)
  --------------
  samples:          2
  evals/sample:     1

### Testowanie funkcji styblinskiTang

In [25]:
J(styblinskiTang, [5.3,2.2,34.2,123.1])

1×4 Matrix{Float64}:
 215.454  -11.404  79458.7  3.72885e6

In [26]:
@benchmark J(styblinskiTang, [rand(5,1)...])

BenchmarkTools.Trial: 
  memory estimate:  4.11 KiB
  allocs estimate:  48
  --------------
  minimum time:     2.415 μs (0.00% GC)
  median time:      2.502 μs (0.00% GC)
  mean time:        2.814 μs (9.18% GC)
  maximum time:     348.425 μs (98.94% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [27]:
@benchmark J(styblinskiTang, [rand(50,1)...])

BenchmarkTools.Trial: 
  memory estimate:  130.81 KiB
  allocs estimate:  561
  --------------
  minimum time:     89.286 μs (0.00% GC)
  median time:      90.426 μs (0.00% GC)
  mean time:        95.550 μs (4.11% GC)
  maximum time:     1.647 ms (94.20% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [28]:
@benchmark J(styblinskiTang, [rand(100,1)...])

BenchmarkTools.Trial: 
  memory estimate:  467.36 KiB
  allocs estimate:  1212
  --------------
  minimum time:     335.781 μs (0.00% GC)
  median time:      347.654 μs (0.00% GC)
  mean time:        368.982 μs (4.28% GC)
  maximum time:     2.679 ms (81.87% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [29]:
@benchmark J(styblinskiTang, [rand(500,1)...])

BenchmarkTools.Trial: 
  memory estimate:  8.20 MiB
  allocs estimate:  7014
  --------------
  minimum time:     7.804 ms (0.00% GC)
  median time:      8.038 ms (0.00% GC)
  mean time:        8.430 ms (2.70% GC)
  maximum time:     17.484 ms (12.45% GC)
  --------------
  samples:          593
  evals/sample:     1

In [30]:
@benchmark J(styblinskiTang, [rand(1000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  32.11 MiB
  allocs estimate:  15018
  --------------
  minimum time:     30.328 ms (0.00% GC)
  median time:      31.550 ms (2.35% GC)
  mean time:        32.103 ms (2.08% GC)
  maximum time:     41.757 ms (0.00% GC)
  --------------
  samples:          156
  evals/sample:     1

In [31]:
@benchmark J(styblinskiTang, [rand(5000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  1.23 GiB
  allocs estimate:  90024
  --------------
  minimum time:     782.028 ms (2.05% GC)
  median time:      814.281 ms (2.32% GC)
  mean time:        817.877 ms (2.26% GC)
  maximum time:     882.285 ms (2.22% GC)
  --------------
  samples:          7
  evals/sample:     1

In [32]:
@benchmark J(styblinskiTang, [rand(10000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  4.89 GiB
  allocs estimate:  190025
  --------------
  minimum time:     3.123 s (2.11% GC)
  median time:      3.162 s (2.35% GC)
  mean time:        3.162 s (2.35% GC)
  maximum time:     3.201 s (2.58% GC)
  --------------
  samples:          2
  evals/sample:     1

## Testy funkcji wektorowych

In [33]:
J(TestFunction5, [5.3,3.3])

5×2 Matrix{Float64}:
  3.13652e13    2.13699e11
  4.71209e169   1.14021
  2.08655e25    0.0274515
 42.4           4.08665
 43.4          -1.54086

In [34]:
@benchmark J(TestFunction1, [rand(2,1)...])

BenchmarkTools.Trial: 
  memory estimate:  1.50 KiB
  allocs estimate:  19
  --------------
  minimum time:     1.968 μs (0.00% GC)
  median time:      2.051 μs (0.00% GC)
  mean time:        2.162 μs (2.59% GC)
  maximum time:     284.724 μs (99.07% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [35]:
@benchmark J(TestFunction2, [rand(2,1)...])

BenchmarkTools.Trial: 
  memory estimate:  2.39 KiB
  allocs estimate:  37
  --------------
  minimum time:     3.287 μs (0.00% GC)
  median time:      3.472 μs (0.00% GC)
  mean time:        3.819 μs (4.49% GC)
  maximum time:     512.399 μs (98.80% GC)
  --------------
  samples:          10000
  evals/sample:     8

In [36]:
@benchmark J(viennet, [rand(2,1)...])

BenchmarkTools.Trial: 
  memory estimate:  2.72 KiB
  allocs estimate:  41
  --------------
  minimum time:     2.110 μs (0.00% GC)
  median time:      2.240 μs (0.00% GC)
  mean time:        2.740 μs (6.23% GC)
  maximum time:     465.395 μs (98.82% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [37]:
@benchmark J(TestFunction4, [rand(2,1)...])

BenchmarkTools.Trial: 
  memory estimate:  2.98 KiB
  allocs estimate:  45
  --------------
  minimum time:     8.030 μs (0.00% GC)
  median time:      8.316 μs (0.00% GC)
  mean time:        8.572 μs (1.00% GC)
  maximum time:     870.266 μs (98.78% GC)
  --------------
  samples:          10000
  evals/sample:     3

In [38]:
@benchmark J(TestFunction5, [rand(2,1)...])

BenchmarkTools.Trial: 
  memory estimate:  3.31 KiB
  allocs estimate:  49
  --------------
  minimum time:     4.956 μs (0.00% GC)
  median time:      5.230 μs (0.00% GC)
  mean time:        5.549 μs (2.82% GC)
  maximum time:     405.443 μs (98.47% GC)
  --------------
  samples:          10000
  evals/sample:     6